In [49]:
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
from gensim.models import KeyedVectors
 
warnings.filterwarnings(action = 'ignore')
 
import gensim
from gensim.models import Word2Vec


import numpy as np
import pandas as pd
import json
import pickle
import heapq
import matplotlib.pyplot as plt
from collections import Counter
from scipy.stats import norm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score
from nltk.tokenize import TweetTokenizer
import re
import sys
import random
import heapq
import ipynb.fs  
sys.path.append("../")
from .defs.get_abstract_2 import count_shared_papers

from sklearn.decomposition import PCA
from sklearn.svm import SVC

In [9]:
with open('../MADStat-dataset-final-version/data.json') as json_file:
    data = json.load(json_file)
    
'''load list of authors'''
with open('../author_name.txt') as f:
    authors = f.readlines()
authors = [author.strip() for author in authors]

'''load papers info'''
papers = pd.read_csv("../paper.csv")

"""load list of authors having at least 30 papers"""
with open("../../authors","rb") as fp:
    author_l = pickle.load(fp)

In [12]:
def clean_text(data) :
        #data.text = data.text.apply(remove_hexa_symbols)
        #data.text = data.text.apply(remove_digits)
        data = data.filter(['author', 'title', 'text']).rename(columns = {'title' : 'doc_id'})
        data["len"] = data.text.apply(lambda x: len(x))
        data.text = data.text.apply(lambda x: re.sub("All rights","",x))
        data.text = data.text.apply(lambda x: re.sub("reserved","",x))
#         data.text = data.text.apply(lambda x: re.sub("[0-9]","",x))
        data.text = data.text.apply(lambda x: re.sub("[^A-Za-z ]","",x))
        data.text = data.text.apply(lambda x: re.sub("copyright","",x))
        data.text = data.text.apply(lambda x: x.lower())
        data = data.loc[data.len > 10].reset_index()
        data.drop(columns=["len"],inplace=True)
        return data
    
def topKFrequent(nums, k):
    dic=Counter(nums)
    heapmax=[[-freq,num] for num,freq in dic.items()]
    heapq.heapify(heapmax)
    list1=[]
    for i in range(k):
        poping=heapq.heappop(heapmax)
        list1.append(poping[1])
    return list1


def get_vocab(text, max_length=200):
#     clf = CountVectorizer(lowercase=True)
#     clf.fit([text])
#     vocab = list(clf.vocabulary_.keys())
#     print("vocab before = ",vocab)
    vocab_f = []
    vocab = text.split()
    for word in set(vocab):
        if word in model:
            vocab_f.append(word)
    return vocab_f

In [169]:
for _ in range(100):
    author1 = random.choice(author_l)
    author2 = random.choice(author_l)
    
    author_1 = pd.read_csv(f'../Data/{author1}.csv').filter(['author', 'title', 'text'])
    author_2 = pd.read_csv(f'../Data/{author2}.csv').filter(['author', 'title', 'text'])
    n, m = author_1.shape[0], author_2.shape[0]
    if author1 != author2 and count_shared_papers(author1,author2,authors,data)==0 and min(n/m, m/n) >= 1/2:   
        data_ = pd.concat([clean_text(author_1),
                                          clean_text(author_2)], ignore_index=True)

        data_train = data_.sample(frac=0.7)
        data_test = data_.drop(data_train.index)
        vocab = get_vocab(''.join([doc + " " for doc in list(data_train["text"])]), max_length=400)


        text1 = data_train[data_train["author"]==author1]
        text2 = data_train[data_train["author"]==author2]



        n, m = text1.shape[0], text2.shape[0]
        embed = np.zeros((n+m,300))
        for i in range(n):
            embed[i,:] = doc_to_vec(text1.text.iloc[i],vocab)
        for j in range(n,n+m):
            embed[j,:] = doc_to_vec(text2.text.iloc[j-n],vocab)
            
        pca = PCA(n_components=10)
        X_train = pca.fit_transform(embed)
        y_train = np.concatenate((np.ones(n),np.zeros(m)))
        
        
        t1 = data_test[data_test["author"]==author1]
        t2 = data_test[data_test["author"]==author2]

        nt, mt = t1.shape[0], t2.shape[0]
        embed_t = np.zeros((nt+mt,300))
        for i in range(nt):
            embed_t[i,:] = doc_to_vec(t1.text.iloc[i],vocab)
        for j in range(nt,nt+mt):
            embed_t[j,:] = doc_to_vec(t2.text.iloc[j-nt],vocab)
            
        X_test = pca.transform(embed_t)
        y_test = np.concatenate((np.ones(nt),np.zeros(mt)))
        y_pred = clf.predict(X_test)
        score = clf.score(X_test,y_test)
        f1 = f1_score(y_pred,y_test)
        if score <= 0.6:
            print(f"Accuracy of {author1} versus {author2} is {score} with f1 score {f1}")

Accuracy of Reinaldo B. Arellano-valle versus Howell Tong is 0.23076923076923078 with f1 score 0.23076923076923078
Accuracy of Hammou El Barmi versus James O. Ramsay is 0.21052631578947367 with f1 score 0.28571428571428564
Accuracy of Amir Dembo versus John Crowley is 0.42857142857142855 with f1 score 0.5384615384615384
Accuracy of Jan Beirlant versus Gang 2 Li is 0.55 with f1 score 0.64
Accuracy of Paul Yip versus Heng Lian is 0.3157894736842105 with f1 score 0.3157894736842105
Accuracy of Michael Kohler versus Francesca Dominici is 0.2857142857142857 with f1 score 0.2857142857142857
Accuracy of Allan Gut versus Andrew Wood is 0.5909090909090909 with f1 score 0.64
Accuracy of Carlos Matrán versus Runze Li is 0.5333333333333333 with f1 score 0.5625
Accuracy of Ruben Zamar versus Masanobu Taniguchi is 0.5185185185185185 with f1 score 0.5185185185185185
Accuracy of Fabienne Comte versus Theo Gasser is 0.3333333333333333 with f1 score 0.2222222222222222
Accuracy of Mohsen Pourahmadi versu

In [96]:
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [97]:
def doc_to_vec(doc,vocab):
    cur = np.zeros(300)
    i = 0
    for word in doc.split():
        if word in vocab:
            i+=1
            cur+=model[word]
    return cur/i

In [157]:
pca = PCA(n_components=10)
X = pca.fit_transform(embed)
X.shape

(47, 10)

In [158]:
y_train = np.concatenate((np.ones(n),np.zeros(m)))
y_train.shape

(47,)

In [159]:
clf = SVC()
clf.fit(X,y_train)

SVC()

In [160]:
t1 = data_test[data_test["author"]==author1]
t2 = data_test[data_test["author"]==author2]

nt, mt = t1.shape[0], t2.shape[0]
embed_t = np.zeros((nt+mt,300))
for i in range(nt):
    embed_t[i,:] = doc_to_vec(t1.text.iloc[i],vocab)
for j in range(nt,nt+mt):
    embed_t[j,:] = doc_to_vec(t2.text.iloc[j-nt],vocab)

In [161]:
X_test = pca.transform(embed_t)

In [162]:
y_test = np.concatenate((np.ones(nt),np.zeros(mt)))

In [163]:
y_pred = clf.predict(X_test)
clf.score(X_test,y_test)

0.85

In [164]:
f1_score(y_pred,y_test)

0.7692307692307692

In [165]:
np.mean(y_pred==y_test)

0.85